In [1]:
import random
import numpy as np
import pandas as pd 
import networkx as nx 
from tqdm import tqdm
from gensim.models import Word2Vec

from alias import create_alias_tables, alias_sample

In [2]:
def random_walk(G, walk_length=10):
    """
    1. 遍历所有节点，以每个节点为起始节点
    2. 从每个起始节点开始，每次都从该节点的邻居节点中随机采样也给节点
    3. 直到当前采样序列的长度达到指定长度或者当前采样的节点没有邻居节点的就停止当前序列的采样
    """
    # 拿出所有的节点
    nodes_list = list(G.nodes) 

    sentense_list = []
    # 遍历所有节点
    for node in tqdm(nodes_list):
        # 以当前node为起始节点
        tmp_sentense_list = [node]
        
        # 如果当前序列已经达到了指定长度，就需要停止采样
        while len(tmp_sentense_list) < walk_length:
            cur_node = tmp_sentense_list[-1]
            # 获取当前节点的邻居节点
            node_nbrs_list = list(G.neighbors(cur_node))
            if len(node_nbrs_list) > 0:
                # 随机选择一个节点
                tmp_sentense_list.append(random.choice(node_nbrs_list))
            else:
                # 否则当前当前序列也需要到此为止
                break
        sentense_list.append(tmp_sentense_list)
    return sentense_list

In [3]:
# 带权的deepwalk随机游走
def deepwalk_weight(G, walk_length=10):
    """
    1. 遍历图中所有节点，以每个节点为起始节点采样
    2. 用带权邻居采样代替随机采样
    """
    nodes_list = G.nodes
    
    # 处理每个节点邻居节点的采样概率，并转化成alias表的形式
    node_alias_dict = {}
    for node in tqdm(nodes_list):
        nbrs_prob_list = [G[node][nbr].get('weight', 1.0) \
                          for nbr in list(G.neighbors(node))]
        normalized_prob_list = [float(prob) / sum(nbrs_prob_list)\
                               for prob in nbrs_prob_list]
        # 构建alias表
        node_alias_dict[node] = create_alias_tables(normalized_prob_list)
    
    # 遍历每个节点，以每个节点为起始节点进行采样
    sentenses_list = []
    for node in tqdm(nodes_list):
        tmp_sentence_list = [node]
        
        while len(tmp_sentence_list) < walk_length:
            cur_node = tmp_sentence_list[-1]
            nbrs_list = list(G.neighbors(cur_node))
            
            # 如果当前节点没有邻居节点，则停止采样
            if len(nbrs_list) == 0:
                break
            
            # 获取当前节点邻居节点的alias表
            accept_prob = node_alias_dict[cur_node][0]
            alias_table = node_alias_dict[cur_node][1]
            sample_index = alias_sample(accept_prob, alias_table)
            tmp_sentence_list.append(nbrs_list[sample_index])
        
        sentenses_list.append(tmp_sentence_list)
    
    return sentenses_list

In [4]:
params_dict = {
    'sentences': '',
    'vector_size': 32,
    'sg': 1,
    'min_count': 0,
    'window': 10,
    'negative': 5,
    'workers': 10,
    'epochs': 10
}

G = nx.read_edgelist('./graph.csv', create_using=nx.DiGraph(), \
                     nodetype=None, data=[('weight', int)])
sentences = random_walk(G)

params_dict['sentences'] = sentences

# 定义word2vec模型
w2v_model = Word2Vec(**params_dict)

100%|████████████████████████████████████████████████████████████████████████████████████| 31364/31364 [00:00<00:00, 72095.39it/s]


In [5]:
params_dict = {
    'sentences': '',
    'vector_size': 32,
    'sg': 1,
    'min_count': 0,
    'window': 10,
    'negative': 5,
    'workers': 10,
    'epochs': 10
}

G = nx.read_edgelist('./graph.csv', create_using=nx.DiGraph(), \
                     nodetype=None, data=[('weight', int)])
sentences = deepwalk_weight(G)

params_dict['sentences'] = sentences

# 定义word2vec模型
w2v_model = Word2Vec(**params_dict)

100%|████████████████████████████████████████████████████████████████████████████████████| 31364/31364 [00:00<00:00, 61246.85it/s]
